# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Описание данных
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
# Открываю файл и просматриваю общую информацию
import pandas as pd

data = pd.read_csv('/datasets/data.csv')
data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [2]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [3]:
print("Количество строк 'days_employed' с положительными значениями:", data[data['days_employed'] > 0].shape[0])
print("Количество строк 'days_employed' с отрицательными значениями:", data[data['days_employed'] < 0].shape[0])
print("Количество строк с нулевым возрастом:", data[data['dob_years'] == 0].count()[0])
print("Количество строк с -1 ребенком:", data[data['children'] == -1].count()[0])
print("Количество строк с 20 детьми:", data[data['children'] == 20].count()[0])

Количество строк 'days_employed' с положительными значениями: 3445
Количество строк 'days_employed' с отрицательными значениями: 15906
Количество строк с нулевым возрастом: 101
Количество строк с -1 ребенком: 47
Количество строк с 20 детьми: 76


**Вывод** Видим аномальные значения в столбце `days_employed`, а именно отрицательное значение. Также видим разные регистры в `education`. Видим аномальное значение в столбце `children`, где количество детей равно -1. Что касается 20 детей, то такое вполне возможно. Также странно, что минимальный возраст в колонке `dob_years` равен нулю.

Слишком много записей в рабочих для с отрицательными значениями. Высокая вероятность, что это просто ошибка и следует привести их к модулю. 
Данные людей с нулевым возрастом является пропуском. 
-1 привести к модулю
20 детей у 76 семей явная ошибка. Видимо добавили лишний 0, реальное значение равно 2.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
# Считаю количество пропусков в каждом столбце
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
data_nulls = data[(data['total_income'].isnull() == True) & (data['days_employed'].isnull() == True)]['income_type'].value_counts()
data_nulls

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

**Вывод** Посчитали пропуски в каждом столбце. Пропуски рандомны (видим, что клиенты работают в разных сферах, а некоторые из них и вовсе уже пенсионеры), поэтому следует заполнить их медианными значениями.

### Замена типа данных

In [6]:
# Переводим все в числовой формат
data['days_employed'] = pd.to_numeric(data['days_employed'], errors='coerce')
data['total_income'] = pd.to_numeric(data['total_income'], errors='coerce')

# Отрицательные числа "превращаем" в положительные
data['days_employed'] = abs(data['days_employed'])
data['total_income'] = abs(data['total_income'])
data['children'] = abs(data['children'])

# Исправляем прочие аномалии
data['children'] = data['children'].replace(-1, 0)
data['children'] = data['children'].replace(20, 0)

# Заменяем нулевой возраст на медианное значение
data_median_age = data[data['dob_years'] != 0]['dob_years'].mean()
data['dob_years'] = data['dob_years'].replace(0, data_median_age)

# Посчитаем медиану для обоих столбцов
days_employed_median = data['days_employed'].median()
total_income_median = data['total_income'].median()

# Вставим медианные значения вместо пропущенных
data['days_employed'] = data['days_employed'].fillna(days_employed_median)
data['total_income'] = data['total_income'].fillna(total_income_median)

# Переводим следующие столбцы из float в int
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

# Проверяем пропущенные значения
data.isnull().sum()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


In [7]:
# Проверяем аномалии снова
print("Количество строк 'days_employed' с положительными значениями:", data[data['days_employed'] > 0].shape[0])
print("Количество строк 'days_employed' с отрицательными значениями:", data[data['days_employed'] < 0].shape[0])
print("Количество строк с нулевым возрастом:", data[data['dob_years'] == 0].count()[0])
print("Количество строк с -1 ребенком:", data[data['children'] == -1].count()[0])
print("Количество строк с 20 детьми:", data[data['children'] == 20].count()[0])

Количество строк 'days_employed' с положительными значениями: 21525
Количество строк 'days_employed' с отрицательными значениями: 0
Количество строк с нулевым возрастом: 0
Количество строк с -1 ребенком: 0
Количество строк с 20 детьми: 0


**Вывод** Посчитали медианное значение и заменили ими пропущенные значения в days_employed и total_income. Также эти же столбцы перевели в int. Исправили все аномальные значения.

### Обработка дубликатов

In [8]:
# Переводим все в нижний регистр и считаем количество дубликатов
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()

data.duplicated().sum()

71

In [9]:
# Удаляем дубликаты и снова проводим проверку на количество дубликатов

data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

**Вывод** Перевели всё в нижний регистр и удалили дубликаты со сбросом текущего индекса.


### Лемматизация

In [10]:
# Импортируем библиотеку для леммитизации целей кредитования
# Создаем функцию и используя метод apply лемметизируем столбец Purpose

from pymystem3 import Mystem
m = Mystem()

def lemma(lem):
    new_lem = m.lemmatize(lem)
    return new_lem
    
data['purpose_lemma'] = data['purpose'].apply(lemma)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43.0,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]"
21450,0,343937,67.0,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]"
21451,1,2113,38.0,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]"
21452,3,3112,38.0,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]"


**Вывод** Провели леммитизацию целей кредитования. В дальнейшем это поможет категоризировать цели получения кредита.

### Категоризация данных

In [11]:
# Категоризируем леммитизированный столбец в новом столбце
def purpose(lem):
    if 'автомобиль' in lem:
        return 'авто'
    elif 'свадьба' in lem:
        return 'свадьба'
    elif 'недвижимость' in lem:
        return 'недвижимость'
    elif 'жилье' in lem:
        return 'жилье'
    elif 'образование' in lem:
        return 'образование'
    else:
        return 'другое'
    
data['category_purpose'] = data['purpose_lemma'].apply(purpose)

median_income = data['total_income'].median()
min_income = data['total_income'].min()
max_income = data['total_income'].max()

print(f'Минимальный доход: {int(min_income)}\nМаксимальный доход: {int(max_income)}\nМедианный доход: {int(median_income)}\n')

def income(i):
    if i < 60000:
        return 'низкие доходы'
    elif 60000 < i < 120000:
        return 'средние доходы'
    else:
        return 'высокие доходы'
    
data['income_rate'] = data['total_income'].apply(income)

def age(i):
    if i < 35: #11 ноября 2020 года Государственная дума одобрила проект о повышении возраста молодёжи до 35 лет (с 14 до 35 лет).
        return 'молодёжь'
    elif 35 < i < 60:
        return 'взрослые'
    else:
        return 'пенсионеры'
    
data['age_status'] = data['dob_years'].apply(age)
data

Минимальный доход: 20667
Максимальный доход: 2265604
Медианный доход: 145017



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,category_purpose,income_rate,age_status
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",жилье,высокие доходы,взрослые
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",авто,средние доходы,взрослые
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",жилье,высокие доходы,молодёжь
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,высокие доходы,молодёжь
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,высокие доходы,взрослые
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43.0,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]",жилье,высокие доходы,взрослые
21450,0,343937,67.0,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]",авто,высокие доходы,пенсионеры
21451,1,2113,38.0,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]",недвижимость,средние доходы,взрослые
21452,3,3112,38.0,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]",авто,высокие доходы,взрослые


**Вывод** Провел категоризацию по целям кредитования. Посчитал минимальный, максимальный и медианные доходы. Условно разделил доходы на высокие, средние и низкие. Взял данные из открытых источников по доходам в <a href="https://www.rbc.ru/economics/18/06/2018/5b27bf829a794710796bbfe5">России</a>. Добавили возрастную категоризацию, для того, чтобы сделать удобные выводы по возрастной категории.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [12]:
data.groupby('children')['debt'].mean()

children
0    0.075598
1    0.091658
2    0.094542
3    0.081818
4    0.097561
5    0.000000
Name: debt, dtype: float64

**Вывод** Как видим из данных, клиенты у которых нет детей имеют меньше задолженностей (7.54%). Есть предположение, что из-за отсутствия детей такие клиенты имеют больше свободных денег, что сказывается и на выплате долгов.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [13]:
data.groupby('family_status')['debt'].mean()

family_status
в разводе                0.071130
вдовец / вдова           0.065693
гражданский брак         0.093471
женат / замужем          0.075452
не женат / не замужем    0.097509
Name: debt, dtype: float64

In [14]:
# Находим медианный возраст и медианные доходы вдов и вдовцов
widows = data[data['family_status'] == 'вдовец / вдова']
widows_median_age = widows['dob_years'].median()
widows_median_income = widows['total_income'].median()
print(f'Медианные доходы вдов и вдовцов: {int(widows_median_income)}\nМедианный возраст вдов и вдовцов: {int(widows_median_age)}')

Медианные доходы вдов и вдовцов: 136897
Медианный возраст вдов и вдовцов: 58


In [15]:
# Находим медианный возраст и медианные доходы не одиночек
single = data[data['family_status'] == 'не женат / не замужем']
single_median_age = single['dob_years'].median()
single_median_income = single['total_income'].median()
print(f'Медианные доходы одиночек: {int(single_median_income)}\nМедианный возраст одиночек: {int(single_median_age)}')

Медианные доходы одиночек: 145017
Медианный возраст одиночек: 36


**Вывод** Из таблицы видим, что вдовцы/вдовы лучше оплачивают задолженность, хоть их доходы в медиане и ниже, чем у одиночек на 9 тысяч. Предполагаем, что из-за возраста вдовцы/вдовы меньше тратят денег на содержание своей жизни, чем молодые люди, которые только пытаются обустроить свою жизнь.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [16]:
data_income = data.pivot_table(index=['income_rate'], values='debt', aggfunc='mean')
data_income['clients_with_debt'] = data.groupby('income_rate')['debt'].sum()
def percent(i):
    return "{0:.2%}".format(i)

data_income['debt'] = data_income['debt'].apply(percent)
data_income = data_income.rename(columns={"debt": "mean, %"})
data_income

,"mean, %",clients_with_debt
income_rate,,
высокие доходы,8.15%,1190
низкие доходы,6.08%,49
средние доходы,8.31%,502


**Вывод** Предполагается, что клиенты с низким доходам очень тщательно следят за кредитной историей, чтобы в следующий раз получить кредит. Клиенты с высоким и средним доходам меньше следят за своей кредитной историей, поэтому в среднем они больше имеют задолженностей по кредиту.

- Как разные цели кредита влияют на его возврат в срок?

In [17]:
purpose_data = data.pivot_table(index=['category_purpose'], values='debt', aggfunc='mean')
purpose_data['clients_with_debt'] = data.groupby('category_purpose')['debt'].sum()
def percent(i):
    return "{0:.2%}".format(i)

purpose_data['debt'] = purpose_data['debt'].apply(percent)
purpose_data = purpose_data.rename(columns={"debt": "mean, %"})
purpose_data

,"mean, %",clients_with_debt
category_purpose,,
авто,9.36%,403
жилье,6.91%,308
недвижимость,7.46%,474
образование,9.22%,370
свадьба,8.00%,186


**Вывод** Лучшие показатели по возврату долгов в разделе "жилье". Наименее ответственные клиенты в категории авто.

## Шаг 4. Общий вывод

**Выводы**
1. Дети: Лучшие клиенты взрослые (от 35 до 60 лет) с 5 детьми. Хуже всех возвращают кредиты пенсионеры с 3 детьми.
2. Семейный статус: Лучшие клиенты замужние/женатые пенсионеры. Худшие - не женатые/не замужние молодежь.
3. Доходы: Пенсионеры со средним доходом. Худшие - молодежь со средним доходом.
4. Цель кредитования: Лучшие клиенты пенсионеры получающие кредит с целью покупки недвижимости. Худшими по возвращаемости кредитов является молодежь, берущая займ на образование.

Пс. Не стал учитывать сегменты клиентов, в которых меньше 50 человек (считаю, что нужно больше данных по таким клиентам, чтобы объективно сравнивать с другими сегментами).

In [18]:
children = data.pivot_table(index=['children', 'age_status'], values='debt', aggfunc='mean')

def percent(i):
    return "{0:.2%}".format(i)

children['clients_with_debt'] = data.groupby(['children','age_status'])['debt'].sum()
children['clients_overall'] = data.groupby(['children','age_status'])['debt'].count()
children['debt'] = children['debt'].apply(percent)
children = children.rename(columns={'debt':'debt, %'})
children

debt, %  clients_with_debt  clients_overall
children age_status                                            
0        взрослые     7.09%                602             8494
         молодёжь    10.85%                335             3088
         пенсионеры   5.18%                134             2585
1        взрослые     8.25%                220             2666
         молодёжь    11.11%                202             1818
         пенсионеры   6.20%                 23              371
2        взрослые     8.07%                 82             1016
         молодёжь    11.26%                101              897
         пенсионеры   7.91%                 11              139
3        взрослые     8.24%                 14              170
         молодёжь     7.69%                 11              143
         пенсионеры  11.76%                  2               17
4        взрослые    10.53%                  2               19
         молодёжь    10.00%                  2               20
         пенсионеры   0.00%                  0                2
5        взрослые     0.00%                  0                6
         молодёжь     0.00%                  0                1
         пенсионеры   0.00%                  0                2

In [19]:
family_status = data.pivot_table(index=['family_status', 'age_status'], values='debt', aggfunc='mean')

def percent(i):
    return "{0:.2%}".format(i)

family_status['clients_with_debt'] = data.groupby(['family_status', 'age_status'])['debt'].sum()
family_status['clients_overall'] = data.groupby(['family_status', 'age_status'])['debt'].count()
family_status['debt'] = family_status['debt'].apply(percent)
family_status = family_status.rename(columns={'debt':'debt, %'})
family_status

debt, %  clients_with_debt  clients_overall
family_status         age_status                                            
в разводе             взрослые     6.19%                 50              808
                      молодёжь    11.44%                 23              201
                      пенсионеры   6.45%                 12              186
вдовец / вдова        взрослые     6.33%                 34              537
                      молодёжь    11.76%                  2               17
                      пенсионеры   6.67%                 27              405
гражданский брак      взрослые     9.26%                219             2365
                      молодёжь    11.14%                140             1257
                      пенсионеры   5.48%                 29              529
женат / замужем       взрослые     6.90%                516             7473
                      молодёжь    10.30%                327             3175
                      пенсионеры   5.20%                 88             1691
не женат / не замужем взрослые     8.50%                101             1188
                      молодёжь    12.07%                159             1317
                      пенсионеры   4.59%                 14              305

In [20]:
monthly_income = data.pivot_table(index=['income_rate', 'age_status'], values='debt', aggfunc='mean')

def percent(i):
    return "{0:.2%}".format(i)

monthly_income['clients_with_debt'] = data.groupby(['income_rate', 'age_status'])['debt'].sum()
monthly_income['clients_overall'] = data.groupby(['income_rate', 'age_status'])['debt'].count()
monthly_income['debt'] = monthly_income['debt'].apply(percent)
monthly_income = monthly_income.rename(columns={'debt':'debt, %'})
monthly_income

debt, %  clients_with_debt  clients_overall
income_rate    age_status                                            
высокие доходы взрослые     7.40%                635             8586
               молодёжь    10.76%                446             4144
               пенсионеры   5.80%                109             1879
низкие доходы  взрослые     5.16%                 23              446
               молодёжь     9.46%                 14              148
               пенсионеры   5.66%                 12              212
средние доходы взрослые     7.85%                262             3339
               молодёжь    11.40%                191             1675
               пенсионеры   4.78%                 49             1025

In [21]:
main_purpose = data.pivot_table(index=['category_purpose', 'age_status'], values='debt', aggfunc='mean')

def percent(i):
    return "{0:.2%}".format(i)

main_purpose['clients_with_debt'] = data.groupby(['category_purpose', 'age_status'])['debt'].sum()
main_purpose['clients_overall'] = data.groupby(['category_purpose', 'age_status'])['debt'].count()
main_purpose['debt'] = main_purpose['debt'].apply(percent)
main_purpose = main_purpose.rename(columns={'debt':'debt, %'})
main_purpose

debt, %  clients_with_debt  clients_overall
category_purpose age_status                                            
авто             взрослые     8.33%                208             2496
                 молодёжь    12.77%                150             1175
                 пенсионеры   7.09%                 45              635
жилье            взрослые     6.21%                158             2546
                 молодёжь     9.28%                119             1283
                 пенсионеры   4.91%                 31              631
недвижимость     взрослые     7.08%                262             3703
                 молодёжь     9.99%                174             1742
                 пенсионеры   4.19%                 38              906
образование      взрослые     8.02%                184             2295
                 молодёжь    13.07%                145             1109
                 пенсионеры   6.73%                 41              609
свадьба          взрослые     8.11%                108             1331
                 молодёжь     9.57%                 63              658
                 пенсионеры   4.48%                 15              335